In [1]:
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import os

In [2]:
path = os.getcwd()
pdbs = ['APD3', 'CancerPPD', 'DBBASP', 'FermFooDb', 'MAHMI', 'PepBDB', 'THPdb']

In [3]:
dfs = {}
for pdb in pdbs:
    dfs[pdb] = pd.read_csv(f'{path}/PDBs/{pdb}.csv')

In [4]:
count_id = 1
data = {
    'LBQCPDB ID': [],
    'Source ID': [],
    'Sequence': [],
    'Activity': [],
    'Source PDB': [],
    'DOI': []
}

In [5]:
# APD3
for i in tqdm(range(0, len(dfs['APD3']))):
    line = dfs['APD3'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['APD ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append(line['Activity'])
    data['Source PDB'].append('APD3')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 3579/3579 [00:00<00:00, 7321.47it/s]


In [6]:
# CancerPPD
for i in tqdm(range(0, len(dfs['CancerPPD']))):
    line = dfs['CancerPPD'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append('Not Found')
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('CancerPPD')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 5068/5068 [00:00<00:00, 8750.22it/s]


In [7]:
# DBBASP
for i in tqdm(range(0, len(dfs['DBBASP']))):
    line = dfs['DBBASP'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('DBBASP')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 20420/20420 [00:02<00:00, 9121.54it/s]


In [8]:
# FermFooDb
for i in tqdm(range(0, len(dfs['FermFooDb']))):
    line = dfs['FermFooDb'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['Link'].split('=')[1])
    data['Sequence'].append(line['Peptide Sequence'])
    data['Activity'].append(line['Activity'])
    data['Source PDB'].append('FermFooDb')
    data['DOI'].append(line['DOI'])
    count_id += 1

100%|██████████| 2205/2205 [00:00<00:00, 7975.96it/s]


In [9]:
# MAHMI
for i in tqdm(range(0, len(dfs['MAHMI']))):
    line = dfs['MAHMI'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['SEQUENCE'])
    data['Activity'].append(line['BIOACTIVITY'])
    data['Source PDB'].append('MAHMI')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 276400/276400 [00:30<00:00, 9138.58it/s]


In [10]:
# PepBDB
for i in tqdm(range(0, len(dfs['PepBDB']))):
    line = dfs['PepBDB'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['Peptide ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('PepBDB')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 13301/13301 [00:01<00:00, 8652.20it/s]


In [11]:
# THPdb
for i in tqdm(range(0, len(dfs['THPdb']))):
    line = dfs['THPdb'].loc[i]
    data['LBQCPDB ID'].append('LBQCPDB_{:6}'.format(count_id).replace(' ', '0'))
    data['Source ID'].append(line['ID'])
    data['Sequence'].append(line['Sequence'])
    data['Activity'].append('Not Found')
    data['Source PDB'].append('THPdb')
    data['DOI'].append('Not Found')
    count_id += 1

100%|██████████| 852/852 [00:00<00:00, 8869.96it/s]


In [12]:
df = pd.DataFrame(data)

In [13]:
dt = datetime.now()
df.to_csv(f'{path}/PDBs/LBQCPDB_releases/LBQCPDB_{dt.strftime("%Y_%m_%d")}.csv', index=False)
df.to_csv(f'{path}/LBQCPDB.csv', index=False)

In [14]:
df.describe()

,LBQCPDB ID,Source ID,Sequence,Activity,Source PDB,DOI
count,321825,321825,321710,320682,321825,321825
unique,321825,315908,304878,407,7,5088
top,LBQCPDB_000001,Not Found,GIVEQCCTSICSLYQLENYCN,Immunomodulatory,MAHMI,Not Found
freq,1,5068,244,183169,276400,296008


In [20]:
df.loc[df['Activity'] == 'Immunomodulatory']

,LBQCPDB ID,Source ID,Sequence,Activity,Source PDB,DOI
29201,LBQCPDB_029202,FMDB135,LYQEPVLGPVRGPFPIIV,Immunomodulatory,FermFooDb,https://doi.org/10.1007/s00253-016-7414-y
29202,LBQCPDB_029203,FMDB136,LYQEPVLGPVRGPFPIIV,Immunomodulatory,FermFooDb,https://doi.org/10.1007/s00253-016-7414-y
29203,LBQCPDB_029204,FMDB137,LYQEPVLGPVRGPFPIIV,Immunomodulatory,FermFooDb,https://doi.org/10.1007/s00253-016-7414-y
29204,LBQCPDB_029205,FMDB138,LYQEPVLGPVRGPFPIIV,Immunomodulatory,FermFooDb,https://doi.org/10.1007/s00253-016-7414-y
29258,LBQCPDB_029259,FMDB192,QEPVPDPVRGLHP,Immunomodulatory,FermFooDb,Not Found
...,...,...,...,...,...,...
307667,LBQCPDB_307668,37790208,SGDITCYLGAV,Immunomodulatory,MAHMI,Not Found
307668,LBQCPDB_307669,36108688,QGTNCIRIRCY,Immunomodulatory,MAHMI,Not Found
307669,LBQCPDB_307670,33623027,MLSIIIPLYNK,Immunomodulatory,MAHMI,Not Found
307670,LBQCPDB_307671,29597671,TVRRESLELPNGNRIPDY,Immunomodulatory,MAHMI,Not Found
